# Converting SPEI Values to Admin 1 and Admin 2

In [1]:
%load_ext jupyter_black
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

In [3]:
data_dir = Path(os.getenv("HAM_DIR")) / "evapotranspiration/"
shp_dir = Path(os.getenv("AA_DATA_DIR")) / "public/raw/ner/cod_ab/ner_cod_ab/"

In [4]:
ner_shp_adm0 = gpd.read_file(shp_dir / "adm0.shp.zip")
ner_shp_adm1 = gpd.read_file(shp_dir / "adm1.shp.zip")
ner_shp_adm2 = gpd.read_file(shp_dir / "adm2.shp.zip")

In [5]:
ner_shp_adm0.total_bounds

array([ 0.16625  , 11.69697  , 15.99564  , 23.5331862])

## Admin 1

In [5]:
ner_shp_adm1.drop(
    ["OBJECTID", "Shape_Leng", "Shape_Area", "ISO3", "ISO2"], axis=1, inplace=True
)
ner_shp_adm1

,adm_01,rowcacode1,NOMREG,geometry
0,Agadez,NER001,AGADEZ,"POLYGON ((11.98901 23.53319, 11.99796 23.51602..."
1,Diffa,NER002,DIFFA,"POLYGON ((15.55942 18.00625, 15.55675 17.95649..."
2,Dosso,NER003,DOSSO,"POLYGON ((3.66840 11.98401, 3.66388 11.98413, ..."
3,Maradi,NER004,MARADI,"POLYGON ((7.76947 14.42108, 7.82971 14.41669, ..."
4,Niamey,NER008,NIAMEY,"POLYGON ((2.09448 13.61969, 2.09711 13.61829, ..."
5,Tahoua,NER005,TAHOUA,"POLYGON ((4.87129 18.67932, 4.83798 18.58273, ..."
6,Tillabéri,NER006,TILLABERI,"POLYGON ((3.88129 15.68909, 4.01208 15.39410, ..."
7,Zinder,NER007,ZINDER,"POLYGON ((12.00488 17.41852, 12.00427 15.99390..."


In [19]:
file_list = [file for file in os.listdir(data_dir) if file.endswith(".tif")]

adm1_spei_df = pd.DataFrame()
for file in file_list:
    file_path = Path(data_dir / file)
    input_raster = rasterio.open(file_path)
    array = input_raster.read(1)
    summary_stats = zonal_stats(
        ner_shp_adm1,
        array,
        stats=["mean", "median"],
        nodata=0,
        all_touched=True,
        affine=input_raster.transform,
    )
    adm1_stats = pd.DataFrame(summary_stats)
    file_df = pd.merge(
        ner_shp_adm1[["rowcacode1", "NOMREG"]],
        adm1_stats,
        left_index=True,
        right_index=True,
    )
    file_df["date"] = file[(file.find("speimax.") + 10) : file.find(".tif")]
    adm1_spei_df = pd.concat([adm1_spei_df, file_df], axis=0)

In [20]:
adm1_spei_df

,rowcacode1,NOMREG,mean,median,date
0,NER001,AGADEZ,21.028435,20.659351,1983.01
1,NER002,DIFFA,23.246147,22.567726,1983.01
2,NER003,DOSSO,28.561845,28.470121,1983.01
3,NER004,MARADI,25.613344,25.586445,1983.01
4,NER008,NIAMEY,27.571728,27.573540,1983.01
...,...,...,...,...,...
3,NER004,MARADI,32.801089,32.797577,2016.12
4,NER008,NIAMEY,34.334667,34.330894,2016.12
5,NER005,TAHOUA,32.743679,32.644085,2016.12
6,NER006,TILLABERI,33.882229,33.753651,2016.12


In [21]:
adm1_spei_df.to_csv(data_dir / "ner_adm1_spei.csv", index=False)

## Admin 2

In [22]:
ner_shp_adm2.drop(
    ["OBJECTID", "Shape_Leng", "Shape_Area", "adm_01", "rowcacode1", "ISO3", "ISO2"],
    axis=1,
    inplace=True,
)
ner_shp_adm2

,adm_02,rowcacode2,NOMDEP,geometry
0,Abala,NER006001,ABALA,"POLYGON ((3.88129 15.68909, 4.01208 15.39410, ..."
1,Abalak,NER005001,ABALAK,"POLYGON ((6.60449 16.28528, 6.67108 16.12830, ..."
2,Aderbissinat,NER001001,ADERBISSINAT,"POLYGON ((10.98339 17.64841, 10.99670 17.10229..."
3,Aguié,NER004001,AGUIE,"POLYGON ((7.78027 13.70728, 7.79572 13.69031, ..."
4,Arlit,NER001002,ARLIT,"POLYGON ((6.39447 17.90632, 6.19025 17.90357, ..."
...,...,...,...,...
62,Tillabéri,NER006012,TILLABERI,"POLYGON ((1.55408 14.64893, 1.57910 14.61090, ..."
63,Tillia,NER005012,TILLIA,"POLYGON ((4.98010 16.32788, 5.04633 16.29150, ..."
64,Torodi,NER006013,TORODI,"POLYGON ((1.50208 13.57770, 1.51208 13.56927, ..."
65,Ville de Tahoua,NER005013,VILLE DE TAHOUA,"POLYGON ((5.24548 15.00671, 5.24688 15.00409, ..."


In [23]:
adm2_spei_df = pd.DataFrame()
for file in file_list:
    file_path = Path(data_dir / file)
    input_raster = rasterio.open(file_path)
    array = input_raster.read(1)
    summary_stats = zonal_stats(
        ner_shp_adm2,
        array,
        stats=["mean", "median"],
        nodata=0,
        all_touched=True,
        affine=input_raster.transform,
    )
    adm2_stats = pd.DataFrame(summary_stats)
    file_df = pd.merge(
        ner_shp_adm2[["rowcacode2", "NOMDEP"]],
        adm2_stats,
        left_index=True,
        right_index=True,
    )
    file_df["date"] = file[(file.find("speimax.") + 10) : file.find(".tif")]
    adm2_spei_df = pd.concat([adm2_spei_df, file_df], axis=0)

In [24]:
adm2_spei_df

,rowcacode2,NOMDEP,mean,median,date
0,NER006001,ABALA,26.922115,26.929800,1983.01
1,NER005001,ABALAK,24.426890,24.380651,1983.01
2,NER001001,ADERBISSINAT,22.163588,22.188341,1983.01
3,NER004001,AGUIE,26.038957,26.060598,1983.01
4,NER001002,ARLIT,23.216956,23.261383,1983.01
...,...,...,...,...,...
62,NER006012,TILLABERI,33.573282,33.575920,2016.12
63,NER005012,TILLIA,32.981634,32.986641,2016.12
64,NER006013,TORODI,34.576988,34.463951,2016.12
65,NER005013,VILLE DE TAHOUA,33.254691,33.274147,2016.12


In [25]:
adm2_spei_df.to_csv(data_dir / "ner_adm2_spei.csv", index=False)